<a href="https://colab.research.google.com/github/nicolastibata/MINE_4210_ADL_202520/blob/main/labs/Laboratorio_9/MINE__4210_ADL_202520_L9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![Logo ADL](https://github.com/nicolastibata/MINE_4210_ADL_202520/blob/main/docs/images/logo.png?raw=true)


# **Laboratorio 9: BERT + Generación de texto**
**Tutor: Nicolás Tibatá**

## **Tabla de Contenido**

[Contexto y objetivos](#scrollTo=5KnQpgpopi8a)<br>
[1. Introducción de los datos](#scrollTo=VjA8zwzJvmeO)<br>
[2. Preparación y Modelamiento](#scrollTo=kG8XHROzvuEH)<br>
[3. Taller 4](#scrollTo=JTKc52_Wvs_N)<br>

### **Contexto y Objetivos**
## Contexto y Objetivos

- Se requiere realizar el análisis de sentimientos de un conjunto de frases del sector financiero como parte de la evaluación del sentimiento del mercado y la reputación de empresas, con el objetivo de tomar decisiones de inversión informadas.

### **Objetivos**
1. Construir una Red Neuronal basada en una arquitectura Transformers para llevar a cabo un análisis de sentimientos, ejemplificando así la aplicación de modelos de procesamiento del lenguaje natural.
2. Generar texto en base a gpt2 de manera de ejemplo.
3. Aplicar de manera combinada ambas herramientas en el taller 4.



**Datos:** [finance-sentence](https://www.kaggle.com/datasets/sbhatti/financial-sentiment-analysis)

### **1. Introducción a los datos**

In [ ]:
!pip install torch -q
!pip install keras-tuner -q
!pip install transformers -q
!pip install "tf-models-official==2.13.*" -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 22.3 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'
import shutil
import pandas as pd
import numpy as np
import seaborn as sns


import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
import keras_tuner as kt

from keras.models import Sequential
from keras.layers import Input, Dense, Dropout, LSTM, Flatten
from transformers import TFBertForSequenceClassification, BertTokenizer

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from google.colab import files

import tensorflow_hub as hub
import tensorflow_text

from google.colab import files
from google.colab import userdata

In [ ]:
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')

!kaggle datasets download -d sbhatti/financial-sentiment-analysis
!unzip "financial-sentiment-analysis.zip"

Dataset URL: https://www.kaggle.com/datasets/sbhatti/financial-sentiment-analysis
License(s): CC0-1.0
  0% 0.00/276k [00:00<?, ?B/s]
100% 276k/276k [00:00<00:00, 127MB/s]
Archive:  financial-sentiment-analysis.zip
  inflating: data.csv                


In [ ]:
# Devidir los datos en train y test
data = pd.read_csv('/content/data.csv', sep=',')
data

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral
...,...,...
5837,RISING costs have forced packaging producer Hu...,negative
5838,Nordic Walking was first used as a summer trai...,neutral
5839,"According shipping company Viking Line , the E...",neutral
5840,"In the building and home improvement trade , s...",neutral


Veamos la distribución de los sentimientos en el dataset

In [ ]:
data["Sentiment"].value_counts()

,count
Sentiment,
neutral,3130
positive,1852
negative,860


### **2. Preparación y Modelamiento**

#### **Preparación**

Pasamos el sentimiento (clase) a un valor entero usando LabelEncoder

In [ ]:
label_encoder = LabelEncoder()
data['Sentiment'] = label_encoder.fit_transform(data['Sentiment'])

etiquetas_unicas = label_encoder.classes_
for valor_numerico, etiqueta_original in enumerate(etiquetas_unicas):
    print(f'Valor numérico: {valor_numerico}, Etiqueta original: {etiqueta_original}')

Valor numérico: 0, Etiqueta original: negative
Valor numérico: 1, Etiqueta original: neutral
Valor numérico: 2, Etiqueta original: positive


In [ ]:
# Divide los datos en entrenamiento y prueba
train, test = train_test_split(data, test_size=0.2, stratify=data['Sentiment'], random_state=42, shuffle = True)

# Ahora divide el conjunto de entrenamiento en entrenamiento y validación
train, val = train_test_split(train, test_size=0.2, stratify=train['Sentiment'], random_state=42, shuffle = True)

print("Tamaño de datos de entrenamiento:", train.shape)
print("Tamaño de datos de validación:", val.shape)
print("Tamaño de datos de prueba:", test.shape)

train

Tamaño de datos de entrenamiento: (3738, 2)
Tamaño de datos de validación: (935, 2)
Tamaño de datos de prueba: (1169, 2)


,Sentence,Sentiment
3097,"Digia will also set up two subsidiaries , Digi...",1
5048,$BBRY Sierra. Has a great cash balance and imp...,2
5727,"Britain's FTSE gains, Land Securities up after...",2
185,The Finnish company sold its UK operation - co...,1
4265,Russian Media Ventures ' minority shareholder ...,1
...,...,...
326,( I&H ) in a move to enhance growth .,2
2821,"In addition , a further 29 employees can be la...",0
4365,"The paper industry 's de-inking sludge , which...",1
1603,$JE LOOKS like we are bouncing. Would be nice...,2


In [ ]:
X_train, X_test, X_val= train['Sentence'], test['Sentence'], val['Sentence']
y_train, y_test, y_val= train['Sentiment'], test['Sentiment'], val['Sentiment']

print("x_train", X_train.shape, " y_train:", y_train.shape)
print("x_test:", X_test.shape, "y_test", y_test.shape)
print("x_val", X_val.shape, "y_val:", y_val.shape)

x_train (3738,)  y_train: (3738,)
x_test: (1169,) y_test (1169,)
x_val (935,) y_val: (935,)


#### **Modelamiento BERT**

 Con el código de las siguientes secciones podemos seleccionar uno de los modelos BERT y su correspondiente modelo de preprocesamiento utilizando TensorFlow Hub. Pueden ver más información [aquí](https://huggingface.co/transformers/v3.0.2/model_doc/bert.html).

In [ ]:
# Cargar los modelos de TensorFlow Hub con trainable=True si deseas fine-tuning
bert_preprocess_model = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3",
    name='preprocessing'
)

bert_model = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/2",
    trainable=False, # Solo como extractor de características
    name='BERT_encoder'
)

print(f'BERT model selected: {bert_model}')
print(f'Preprocess model selected: {bert_preprocess_model}')

BERT model selected: <tensorflow_hub.keras_layer.KerasLayer object at 0x7c40c6150260>
Preprocess model selected: <tensorflow_hub.keras_layer.KerasLayer object at 0x7c40c6399400>


In [ ]:
# Capas Bert
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessing_output = bert_preprocess_model(text_input)
outputs = bert_model(preprocessing_output)

# Capas de nuestra red adicional
#l = tf.keras.layers.Dropout(0.1, name="dropout")
l = tf.keras.layers.Dense(512, activation='sigmoid', name="capaOculta1")(outputs['pooled_output'])
l = tf.keras.layers.Dropout(0.2, name="dropout")(l)
y = tf.keras.layers.Dense(3, activation='softmax', name="output")(l)

# Usamos inputs y outputs para construir el modelo final
model = tf.keras.Model(inputs=[text_input], outputs = [y])

In [ ]:
metrics = ['accuracy']

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=metrics)

In [ ]:
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 preprocessing (KerasLayer)  {'input_type_ids': (None,    0         ['text[0][0]']                
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_word_ids': (None,                                           
                              128)}                                                               
                                                                                            

In [ ]:
model.fit(X_train, y_train,
          validation_data = (X_val, y_val),
          epochs=20,
          #callbacks= EarlyStopping(monitor='val_accuracy', patience=4)
          )

Epoch 1/20
117/117 [==============================] - 31s 126ms/step - loss: 0.8894 - accuracy: 0.5920 - val_loss: 0.7672 - val_accuracy: 0.6439
Epoch 2/20
117/117 [==============================] - 11s 93ms/step - loss: 0.7778 - accuracy: 0.6351 - val_loss: 0.7450 - val_accuracy: 0.6620
Epoch 3/20
117/117 [==============================] - 11s 94ms/step - loss: 0.7509 - accuracy: 0.6541 - val_loss: 0.7325 - val_accuracy: 0.6481
Epoch 4/20
117/117 [==============================] - 11s 95ms/step - loss: 0.7170 - accuracy: 0.6664 - val_loss: 0.7217 - val_accuracy: 0.6439
Epoch 5/20
117/117 [==============================] - 11s 94ms/step - loss: 0.7079 - accuracy: 0.6675 - val_loss: 0.6820 - val_accuracy: 0.6759
Epoch 6/20
117/117 [==============================] - 11s 94ms/step - loss: 0.6936 - accuracy: 0.6841 - val_loss: 0.7041 - val_accuracy: 0.6920
Epoch 7/20
117/117 [==============================] - 11s 93ms/step - loss: 0.6775 - accuracy: 0.6891 - val_loss: 0.8361 - val_accuracy

**Evaluamos nuestro modelo**

In [ ]:
y_pred = model.predict(X_train)
y_pred = np.argmax(y_pred, axis=1)

print(classification_report(y_train, y_pred))

117/117 [==============================] - 9s 70ms/step
              precision    recall  f1-score   support

           0       0.53      0.55      0.54       550
           1       0.80      0.87      0.83      2003
           2       0.79      0.66      0.72      1185

    accuracy                           0.75      3738
   macro avg       0.71      0.69      0.70      3738
weighted avg       0.76      0.75      0.75      3738



In [ ]:
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

print(classification_report(y_test, y_pred))

37/37 [==============================] - 3s 76ms/step
              precision    recall  f1-score   support

           0       0.34      0.35      0.35       172
           1       0.76      0.84      0.80       626
           2       0.70      0.57      0.63       371

    accuracy                           0.68      1169
   macro avg       0.60      0.59      0.59      1169
weighted avg       0.68      0.68      0.68      1169



#### **Generación de Texto**

Utilizamos un modelo GPT transformer para generación de texto, el proceso es similar al uso de BERT

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
input_text = "Once upon a time"
input_ids = tokenizer.encode(input_text, return_tensors="pt")

In [ ]:
output = model.generate(
    input_ids,
    max_length=50,           # Longitud máxima del texto generado
    num_return_sequences=1,  # Cantidad de sentencias generadas
    temperature=0.7,         # Temperatura (más alto = más creativo)
    top_k=50,                # Top 50 de palabras probables
    top_p=0.9,               # Centrado a palabras de alta probabilidad
    repetition_penalty=1.2,  # Reducción de repetición de palabras
    do_sample=True           # Habilita el randomness del texto generado
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [ ]:
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

Once upon a time the entire world was like this. But now we are, as I see it: an unshakable sense of pride and joy that our actions will not be questioned or doubted by any other nation in history because they were chosen to


### **3. Taller 4**


Instrucciones

1. El archivo a presentar debe ser en formato .ipynb o HTML con sus celdas ejecutadas. Celdas sin ejecutar no podrán ser evaluadas.
2. El nombre del archivo debe ser taller_4_{Apellido_Nombre}_{Apellido_Nombre} de cada integrante del equipo.
3. Las entregas solo se hacen a través de Bloque Neón.

------


1. Los resultados del modelo base no son los mejores. Utilice el modelo [Distilbert](https://huggingface.co/transformers/v3.0.2/model_doc/distilbert.html) de Huggingface aplicando padding a los diferentes dataframes (train, val, test). ¿Qué es Distilbert? ¿Mejoran los resultados? ¿El entrenamiento es más rápido? Compare los resultados con el modelo base del laboratorio.
```python
# clue

import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from sklearn.model_selection import train_test_split
# Tokenización
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
X_train = [tokenizer.encode(text, add_special_tokens=True, truncation=True, max_length=128) for text in X_train]
X_val = [tokenizer.encode(text, add_special_tokens=True, truncation=True, max_length=128) for text in X_val]
X_test = [tokenizer.encode(text, add_special_tokens=True, truncation=True, max_length=128) for text in X_test]

# Padding
X_train = tf.keras.preprocessing.sequence.pad_sequences(...)
X_val = tf.keras.preprocessing.sequence.pad_sequences(...)
X_test = tf.keras.preprocessing.sequence.pad_sequences(...)

model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
```

2. Genere sentencias nuevas con el modelo gpt2 o algún otro de preferencia para poder tener datos sintéticos y aumentar la cantidad de sentencias negativas del dataset original (aumentar las sentencias negativas hasta 1200 datos de esa clase). Luego reentrene el modelo del punto 1 (es decir el modelo entrenado con Distilbert) con este nuevo dataset aumentado y documente las métricas de evaluación.


3. Realice fine-tuning al modelo base del laboratorio y compárelo con el modelo del punto 2. ¿Que diferencias nota al realizar este paso adicional?
``` python
# clue

trainable=True,

```
